In [ ]:
import pandas as pd
import requests
import sys
import time

# Matthew's keys
noaa_key = "JOwIFUXjlWBqtstSqsSaKYDMzShAccQe"
usda_key = "8D5EDCA6-733D-3ACA-AF06-BA5BB3938B4F"

# Rechelle's keys
# noaa_key= "FtekNkLHUPsNabCSpEVMXldengLWOCAm"
# usda_key= "6B727F30-94D3-3E2A-9186-6F7EB2F90578"

In [ ]:
import requests

def get_usda_data(usda_key, year):
  # API endpoint
  url = "https://quickstats.nass.usda.gov/api/api_GET/"

  params = {
    "key": usda_key,
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "unit_desc": "BU / ACRE",
    "state_alpha": "MI",
    "year": year,
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "agg_level_desc": "COUNTY",
    "format": "JSON"
  }

  response = requests.get(url, params=params)

  if response.status_code == 200:
    data = response.json()
    return pd.DataFrame(data['data'])
  else:
    print(f"Request failed with status code {response.status_code}")
    return None

In [ ]:
def clean_weather(df,state_ansi,county_ansi):
    # For each day in date column average the TMAX and TMIN into their own columns across all stations
    df["date"] = pd.to_datetime(df["date"])
    df["TMAX"] = df["value"].where(df["datatype"] == "TMAX")
    df["TMIN"] = df["value"].where(df["datatype"] == "TMIN")
    
    df = df.groupby("date").agg({"TMAX": "mean", "TMIN": "mean"}).reset_index()

    # add state and county columns
    df["state_ansi"] = state_ansi
    df["county_ansi"] = county_ansi
    
    return df

In [ ]:
# for each state_ansi county_ansi combo in the USDA data, get the weather data

def get_weather_data(usda_df):
    weather_data = []
    for index, row in usda_df.iterrows():
        state_ansi = row["state_ansi"]
        county_ansi = row["county_ansi"]
        #time.sleep(1)
        weather_raw = get_raw_weather(state_ansi, county_ansi, "2022-05-15T00:00:00", "2022-10-15T00:00:00")
        if weather_raw is not None:
            weather = clean_weather(weather_raw, state_ansi, county_ansi)
            weather_data.append(weather)
    return pd.concat(weather_data)

# weather_data = get_weather_data(usda_df)
# weather_data.to_csv("weather_data.csv", index=False)

In [ ]:
# Count the number of unique items in column county for weather_data
# weather_data["county"].nunique()

In [ ]:
# Define a list of 50 years from 1970 to 2019
# Get USDA data for each year
# Get get_weather_data for each year
# Save the data to a CSV file

years = range(2019, 2022)


# for year in years:
#   print(year)
#   usda_data = get_usda_data(usda_key, year)
#   usda_data.to_csv(f"usda_data_{year}.csv", index=False)
#   weather_data=get_weather_data(usda_data)
#   weather_data.to_csv(f"weather_data_{year}.csv", index=False)

In [ ]:
years = range(2019, 2022)


for year in years:
  print(year)
  usda_data = get_usda_data(usda_key, year)
  usda_data.to_csv(f"data/usda_data_{year}.csv", index=False)
  weather_data=get_weather_data(usda_data)
  weather_data.to_csv(f"data/weather_data_{year}.csv", index=False)

In [ ]:
# GDD calculation function
def calculate_gdd(df, base_temp=50, upper_temp=86):
    """
    Calculate Growing Degree Days (GDD) for corn.
    """
    df['TMAX'] = df['TMAX'].clip(lower=base_temp, upper=upper_temp)
    df['TMIN'] = df['TMIN'].clip(lower=base_temp)
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df['GDD'] = df['TAVG'] - base_temp
    return df

In [ ]:
# First call get_usda_data with key and the year of data you would like
usda_data = get_usda_data(usda_key, 2019)

# Then call get_weather_data with the usda_data data frame
# This function calls get_raw_weather then clean_weather for each county in the usda_data data frame

weather_data = get_weather_data(usda_data)
weather_data = calculate_gdd(weather_data)


